# <b style='color:gold;'> LightLGBM </b>
* Este Jupyter tem como função treinar um `LightLGBM` com os melhores parametros determinados por um `scikit-optimize`

---

Bibliotecas/Módulos

<details>    
<summary>
    <font size="3" color="magenta"><b>Install</b></font>
</summary>
<p>
<ul>
    <li> !pip install lightgbm</li>
    <li> !pip install scikit-optimize </li>

</ul>
</p>

In [1]:
%%capture

!pip install lightgbm scikit-optimize

In [2]:
import logging
import numpy as np
import pandas as pd
import sys
from lightgbm import LGBMClassifier
# from sklearn.externals import joblib
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, matthews_corrcoef, classification_report, f1_score
from skopt import forest_minimize
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier

Configurações dos logs

In [3]:
logger = logging.getLogger("SVM")
logger.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stderr)
formatter = logging.Formatter('%(name)s - %(levelname)s - [+] ------- %(message)s -------') 
handler.setFormatter(formatter)
logger.handlers = [handler]

Cores

In [4]:
RED = "\033[1;31m"
BLUE = "\033[1;34m"
GREEN = "\033[1;32m"
PINK = "\033[1;45m"
HEADER = '\033[95m'
OKBLUE = '\033[94m'
OKCYAN = '\033[96m'
OKGREEN = '\033[92m'
WARNING = '\033[93m'
FAIL = '\033[91m'
ENDC = '\033[0m'
BOLD = '\033[1m'
UNDERLINE = '\033[4m'
MAG = "\033[1;45m"

---

In [5]:
df = pd.read_csv("dados_FEM_MAS_10_voluntarios_s_semanas_15_sinais_noout.csv")

In [6]:
df.sample()

,0,1,2,3,4,5,6,7,8,9,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,label
5535,0,2,-2,1,-2,-10,3,-1,-1,11,...,11,5,4,-6,5,8,9,2,2,1


In [7]:
len(df)

25200

In [8]:
len(df.columns)

2001

---

In [9]:
y = df["label"]

In [10]:
df = df.drop(["label"], axis=1)

In [11]:
from sklearn.decomposition import PCA

In [12]:
pca = PCA(n_components=1000)

In [13]:
pca.fit(df)

PCA(n_components=1000)

In [14]:
%%time

pca_samples = pca.transform(df)

CPU times: user 20.3 s, sys: 2.97 s, total: 23.2 s
Wall time: 1.74 s


In [15]:
ps = pd.DataFrame(pca_samples)
ps.head()

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,-35.068197,23.928768,1.186254,-151.129527,-42.556784,-148.023683,-53.831397,-28.461750,-41.857889,-104.510884,...,-14.290352,19.004321,35.661226,16.523947,5.574728,12.551279,26.042565,-30.094703,-18.285243,27.780576
1,-69.680457,36.845880,47.351802,-121.842658,-24.717552,-113.801059,-37.514149,-47.755131,-13.947044,-92.122796,...,-11.845557,9.300641,20.255011,16.304799,4.366212,10.565927,6.045189,-21.770155,-13.068536,18.437145
2,-94.676600,38.038653,98.432548,-103.621516,-10.274245,-71.540178,-15.104704,-53.717266,24.066586,-97.600882,...,-12.620933,8.253709,4.292358,11.146174,17.726639,7.914150,-2.785102,-14.097777,0.273400,3.965218
3,-115.288910,36.334428,137.247898,-70.874058,-8.118221,-44.962368,-3.156064,-64.584880,46.295866,-89.627326,...,-19.479738,14.939521,-8.124009,22.787313,13.210965,3.827481,-27.397729,-10.864761,15.061757,8.040029
4,-111.539509,45.575378,158.235942,-48.798876,-3.696119,-8.481742,11.270159,-71.759108,48.659401,-69.601917,...,-18.038775,4.586447,-17.087370,6.878407,13.042246,14.247241,-32.473337,-1.387909,29.652418,-3.996342


In [16]:
len(ps)

25200

In [17]:
len(ps.columns)

1000

In [18]:
X = ps

In [19]:
x_treino, x_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=7)

---

### LGBM

Instanciando

In [20]:
%%time

mdl_lgbm = LGBMClassifier(random_state=0, class_weight="balanced", n_jobs=2)
mdl_lgbm.fit(x_treino, y_treino)

CPU times: user 29.8 s, sys: 161 ms, total: 30 s
Wall time: 15.4 s


LGBMClassifier(class_weight='balanced', n_jobs=2, random_state=0)

Tuning do Lgbm

In [21]:
def tune_lgbm(params):
    print(params)
    lr = params[0]
    max_depth = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    n_estimators = params[5]
    
    min_df = params[6]
    ngram_range = (1, params[7])
    
    logger.info('Instanciando')
    mdl_tune = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                         min_child_samples=min_child_samples, subsample=subsample,
                         colsample_bytree=colsample_bytree, bagging_freq=1,n_estimators=n_estimators, random_state=0, 
                         class_weight="balanced", n_jobs=2)
    
    logger.info('Treinando')
    mdl_tune.fit(x_treino, y_treino)
    
    logger.info('Predict')
    p = mdl_tune.predict_proba(x_teste)[:, 1]
    categorias = mdl_tune.predict(x_teste)
    
    print('Métricas: \n \n')
    
    logger.info('Roc')
    roc = roc_auc_score(y_teste, p)
    print(OKBLUE, "Roc \n" + str(roc) + "\n")
    
    logger.info('Acurácia')
    acc = accuracy_score(y_teste, categorias)
    print(OKCYAN, "Acc \n" + str(acc) + "\n")
    
    logger.info("Mcc")
    mcc = matthews_corrcoef(y_teste, categorias)
    print(WARNING, "Mcc \n" + str(mcc) + "\n")
    
    logger.info("F1")
    f1 = f1_score(y_teste, categorias, average="weighted")
    print(GREEN, "F1 \n" + str(f1) + "\n")
    
    logger.info('Class Report')
    resultados = classification_report(y_teste, categorias)
    print(ENDC, "Class Report \n" + resultados + "\n")
    
    return -accuracy_score(y_teste, categorias)


In [22]:
space = [(1e-3, 1e-1, 'log-uniform'), # lr
          (1, 10),                    # max_depth
          (1, 20),                    # min_child_samples
          (0.05, 1.),                 # subsample
          (0.05, 1.),                 # colsample_bytree
          (100,1000),                 # n_estimators
          (1,5),                      # min_df
          (1,5)]                      # ngram_range

[0.009944912110647982, 5, 1, 0.4677107511929402, 0.49263223036174764, 272, 3, 1]

In [23]:
%%time

try:
    res = forest_minimize(tune_lgbm, space, random_state=7, n_random_starts=20, n_calls=50, verbose=1)

except Exception as error:
    logger.warning('Aconteceu algum problema...')
    logger.critical(error)

else:
    logger.info('Ok')
    
finally:
    logger.info('Treino finalizado')

SVM - INFO - [+] ------- Instanciando -------
SVM - INFO - [+] ------- Treinando -------


Iteration No: 1 started. Evaluating function at random point.
[0.002848906260926589, 7, 4, 0.9793117522246534, 0.4828056676618013, 285, 1, 2]


SVM - INFO - [+] ------- Predict -------
SVM - INFO - [+] ------- Roc -------
SVM - INFO - [+] ------- Acurácia -------
SVM - INFO - [+] ------- Mcc -------
SVM - INFO - [+] ------- F1 -------
SVM - INFO - [+] ------- Class Report -------
SVM - INFO - [+] ------- Instanciando -------
SVM - INFO - [+] ------- Treinando -------


Métricas: 
 

 Roc 
0.8524724284910736

 Acc 
0.7289682539682539

 Mcc 
0.5023881063602704

 F1 
0.7169124541026457

 Class Report 
              precision    recall  f1-score   support

           0       0.89      0.52      0.66      2518
           1       0.66      0.93      0.78      2522

    accuracy                           0.73      5040
   macro avg       0.78      0.73      0.72      5040
weighted avg       0.78      0.73      0.72      5040


Iteration No: 1 ended. Evaluation done at random point.
Time taken: 31.6280
Function value obtained: -0.7290
Current minimum: -0.7290
Iteration No: 2 started. Evaluating function at random point.
[0.0013934848991057577, 9, 8, 0.551376555079732, 0.8753613852974182, 491, 5, 1]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


SVM - INFO - [+] ------- Predict -------
SVM - INFO - [+] ------- Roc -------
SVM - INFO - [+] ------- Acurácia -------
SVM - INFO - [+] ------- Mcc -------
SVM - INFO - [+] ------- F1 -------
SVM - INFO - [+] ------- Class Report -------
SVM - INFO - [+] ------- Instanciando -------
SVM - INFO - [+] ------- Treinando -------


...

Iteration No: 49 ended. Search finished for the next optimal point.
Time taken: 311.4548
Function value obtained: -0.7879
Current minimum: -0.7909
Iteration No: 50 started. Searching for the next optimal point.
[0.007507933134993518, 9, 17, 0.0601928108219361, 0.08531344234324861, 571, 1, 5]
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1


SVM - INFO - [+] ------- Predict -------
SVM - INFO - [+] ------- Roc -------
SVM - INFO - [+] ------- Acurácia -------
SVM - INFO - [+] ------- Mcc -------
SVM - INFO - [+] ------- F1 -------
SVM - INFO - [+] ------- Class Report -------


Métricas: 
 

 Roc 
0.8570448205119806

 Acc 
0.7490079365079365

 Mcc 
0.4980812773221593

 F1 
0.7489879653613717

 Class Report 
              precision    recall  f1-score   support

           0       0.75      0.74      0.75      2518
           1       0.75      0.76      0.75      2522

    accuracy                           0.75      5040
   macro avg       0.75      0.75      0.75      5040
weighted avg       0.75      0.75      0.75      5040




SVM - INFO - [+] ------- Ok -------
SVM - INFO - [+] ------- Treino finalizado -------


Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 8.8332
Function value obtained: -0.7490
Current minimum: -0.7909
CPU times: user 2h 40min 22s, sys: 28 s, total: 2h 40min 50s
Wall time: 1h 20min 50s


In [24]:
res.

SyntaxError: invalid syntax (1459325066.py, line 1)

 2 voluntarias
 
 accuracy                           0.79      1536

In [ ]:
res.x